In [1]:
import pandas as pd
import requests 
from bs4 import BeautifulSoup
import itertools
import nltk

r = requests.get('https://jomacanning.mitiendanube.com/fb_catalog.xml')

soup = BeautifulSoup(r.text,'html.parser')

soup

title = soup.find_all('title')

description = soup.find_all('description')

title

catalogo = pd.DataFrame()
for i in title:
    catalogo = pd.concat([catalogo,(pd.DataFrame(i))])

catalogo_desc = pd.DataFrame()
for i in description:
    catalogo_desc = pd.concat([catalogo_desc,(pd.DataFrame(i))])

catalogo.columns = ['title']
catalogo.reset_index(inplace=True)
catalogo.drop('index',axis= 1,inplace=True)

catalogo_desc.columns = ['description']
catalogo_desc.reset_index(inplace=True)
catalogo_desc.drop('index',axis= 1,inplace=True)

catalogo['description'] = catalogo_desc['description']

catalogo.title = [x.lower() for x in catalogo.title]
catalogo.description = [x.lower() for x in catalogo.description]

catalogo_title = [x.lower() for x in catalogo.title]
catalogo_description = [x.lower() for x in catalogo.description]

# tokenizer y lista de las palabras

catalogo['tokenized_title'] = catalogo.apply(lambda row: nltk.word_tokenize(row['title']), axis=1)

tokens_catalogo_title = list(itertools.chain.from_iterable(catalogo['tokenized_title']))
#lst_phrases_catalogo = [phrase.replace(" ","_") for phrase in lst_tokens_catalogo]

# tokenizer y lista de las palabras de olenka

catalogo['tokenized_description'] = catalogo.apply(lambda row: nltk.word_tokenize(row['description']), axis=1)

tokens_catalogo_description = list(itertools.chain.from_iterable(catalogo['tokenized_description']))
#lst_phrases_catalogo = [phrase.replace(" ","_") for phrase in lst_tokens_catalogo]



In [18]:
catalogo.title.to_csv('productos_joma.csv')

C:\Users\maria\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """Entry point for launching an IPython kernel.


In [19]:
catalogo = pd.read_csv('productos_joma.csv')

In [23]:
catalogo.drop('0',axis=1,inplace=True)

catalogo.columns = ['title']

In [37]:
# tokenize 
catalogo['tokenized_title'] = catalogo.apply(lambda row: nltk.word_tokenize(row['title']), axis=1)

In [38]:
kw_list = list(itertools.chain.from_iterable(catalogo['tokenized_title']))

In [39]:
#remove duplicated

kw_list = list(dict.fromkeys(kw_list))

In [40]:
kw_list = kw_list[:]

In [49]:
# aca como que las procesa de distintas formas. La columna kemwywords te agarra las keywords sacando las palabras que no suman nada

In [43]:
import re
import nltk

def preprocess(text):
 
    #Basic cleaning
    text = text.strip()
    text = re.sub(r'[^\w\s]','',text)
    text = text.lower()
 
    #Tokenize single comment:
    tokens = nltk.word_tokenize(text)
 
    return(tokens)

def get_hashtags(text):
    hashtags = re.findall(r"#(\w+)", text)
    return(hashtags)

def tag_tokens(preprocessed_tokens):
    pos = nltk.pos_tag(preprocessed_tokens)
    return(pos)

def get_keywords(tagged_tokens,pos='all'):
 
    if(pos == 'all'):
        lst_pos = ('NN','JJ','VB')
    elif(pos == 'nouns'):
        lst_pos = 'NN'
    elif(pos == 'verbs'):
        lst_pos = 'VB'
    elif(pos == 'adjectives'):
        lst_pos = 'JJ'
    else:
        lst_pos = ('NN','JJ','VB')
 
    keywords = [tup[0] for tup in tagged_tokens if tup[1].startswith(lst_pos)]
 
    return(keywords)

def get_noun_phrases(tagged_tokens):
 
    grammar = "NP: {<DT>?<JJ>*<NN>}"
    cp = nltk.RegexpParser(grammar)
    tree = cp.parse(tagged_tokens)
 
    result = []
    for subtree in tree.subtrees(filter=lambda t: t.label() == 'NP'):
        ###We only take phrases not single words
        if(len(subtree.leaves())>1):
            outputs = [tup[0] for tup in subtree.leaves()]
            outputs = " ".join(outputs)
            result.append(outputs)
 
    return(result)

def execute_pipeline(dataframe):
    # #Get hashtags
    oraciones['hashtags'] = oraciones.apply(lambda x: get_hashtags(x['title']),axis=1)
    #Pre-process
    oraciones['preprocessed'] = oraciones.apply(lambda x: preprocess(x['title']),axis=1)
    #Extract pos
    oraciones['tagged'] = oraciones.apply(lambda x: tag_tokens(x['preprocessed']),axis=1)
    #Extract keywords
    oraciones['keywords'] = oraciones.apply(lambda x: get_keywords(x['tagged'],'all'),axis=1)
    #Extract noun_phrases
    oraciones['noun_phrases'] = oraciones.apply(lambda x: get_noun_phrases(x['tagged']),axis=1)
 
    return(dataframe)
 

In [44]:
df_procesado = execute_pipeline(oraciones)

In [45]:
df_procesado

,title,tokenized_title,hashtags,preprocessed,tagged,keywords,noun_phrases
0,calefactor de patio hongo daewoo,"[calefactor, de, patio, hongo, daewoo]",[],"[calefactor, de, patio, hongo, daewoo]","[(calefactor, NN), (de, IN), (patio, FW), (hon...","[calefactor, hongo, daewoo]",[]
1,estufa garrafera daewoo,"[estufa, garrafera, daewoo]",[],"[estufa, garrafera, daewoo]","[(estufa, NN), (garrafera, NN), (daewoo, NN)]","[estufa, garrafera, daewoo]",[]
2,estufa cuarzo 1200w vertical reforzada,"[estufa, cuarzo, 1200w, vertical, reforzada]",[],"[estufa, cuarzo, 1200w, vertical, reforzada]","[(estufa, NN), (cuarzo, NN), (1200w, CD), (ver...","[estufa, cuarzo, vertical, reforzada]",[vertical reforzada]
3,estufa halogena 1200w con giro 3 tubos gacyf,"[estufa, halogena, 1200w, con, giro, 3, tubos,...",[],"[estufa, halogena, 1200w, con, giro, 3, tubos,...","[(estufa, JJ), (halogena, NN), (1200w, CD), (c...","[estufa, halogena, con, giro, tubos, gacyf]",[estufa halogena]
4,caloventor 1800w azul/naranja (azul),"[caloventor, 1800w, azul/naranja, (, azul, )]",[],"[caloventor, 1800w, azulnaranja, azul]","[(caloventor, NN), (1800w, CD), (azulnaranja, ...","[caloventor, azulnaranja, azul]",[]
5,caloventor 1800w azul/naranja (naranja),"[caloventor, 1800w, azul/naranja, (, naranja, )]",[],"[caloventor, 1800w, azulnaranja, naranja]","[(caloventor, NN), (1800w, CD), (azulnaranja, ...","[caloventor, azulnaranja, naranja]",[]
6,split digital c/visor de temperatura 2000w,"[split, digital, c/visor, de, temperatura, 2000w]",[],"[split, digital, cvisor, de, temperatura, 2000w]","[(split, NN), (digital, JJ), (cvisor, NN), (de...","[split, digital, cvisor]",[digital cvisor]
7,aire acondicionado portatil 3500w frio/calor,"[aire, acondicionado, portatil, 3500w, frio/ca...",[],"[aire, acondicionado, portatil, 3500w, friocalor]","[(aire, NN), (acondicionado, NN), (portatil, N...","[aire, acondicionado, portatil, friocalor]",[]
8,pava electrica acero inox indelplas,"[pava, electrica, acero, inox, indelplas]",[],"[pava, electrica, acero, inox, indelplas]","[(pava, NN), (electrica, NN), (acero, NN), (in...","[pava, electrica, acero, inox, indelplas]",[]
9,calefon termoelectrico enlozado 20l,"[calefon, termoelectrico, enlozado, 20l]",[],"[calefon, termoelectrico, enlozado, 20l]","[(calefon, NN), (termoelectrico, NN), (enlozad...","[calefon, termoelectrico, enlozado]",[]


In [46]:
kw_list = list(itertools.chain.from_iterable(df_procesado['keywords']))

#remove duplicated

kw_list = list(dict.fromkeys(kw_list))

kw_list = kw_list

kw_list

['calefactor',
 'hongo',
 'daewoo',
 'estufa',
 'garrafera',
 'cuarzo',
 'vertical',
 'reforzada',
 'halogena',
 'con',
 'giro',
 'tubos',
 'gacyf',
 'caloventor',
 'azulnaranja',
 'azul',
 'naranja',
 'split',
 'digital',
 'cvisor',
 'aire',
 'acondicionado',
 'portatil',
 'friocalor',
 'pava',
 'electrica',
 'acero',
 'inox',
 'indelplas',
 'calefon',
 'termoelectrico',
 'enlozado',
 'inoxidable',
 'ducha',
 'lorenzetti',
 'generador',
 'monofasico',
 'motomel',
 'monofsico',
 'm2500e',
 'm5500e',
 'm6500e',
 'generdor',
 'm8000e',
 'plane',
 'artelum',
 'aplique',
 'anka',
 'bidireccional',
 'unidireccional',
 'eco',
 'farola',
 'nano',
 'pixy',
 'estaca',
 'gump',
 'dura',
 'i',
 'mini',
 'articulo',
 'telescopy',
 'ar111',
 'petra',
 'nico',
 'lampara',
 'copen',
 'silver',
 'velador',
 'marã',
 'cobre',
 'linea',
 'negro',
 'marisa',
 'virgo',
 'libra',
 'cromo',
 'dan',
 'corvo',
 'cruz',
 'rufo',
 'luciana',
 'monique',
 'misterio',
 'ii',
 'pepita',
 'oval',
 'fonix',
 'nt2101

# busqueda google trends

In [ ]:
# esta busqueda a google trends es una de las varias posibles que se puede hacer.  pytrend.interest_over_time() te devuelve el interes de esa(o esas) palabras a traves del tiempo que le especifiques.
# despues hay otras consultas que se puede hacer para agarrar similar queries, o tendencias por ubicacion geografica.

In [48]:
from pytrends.request import TrendReq
import pandas as pd
import time
from tqdm import tqdm_notebook

startTime = time.time()
pytrend = TrendReq(hl='es-AR', tz=360)

df = pd.DataFrame(kw_list)
df.columns=['keywords']

df2 = df["keywords"].values.tolist()
#df2.remove("keywords")
dataset = []
for x in tqdm_notebook(range(0,len(df2))):
     keywords = [df2[x]]
     pytrend.build_payload(
     kw_list=keywords,
     cat=0,
     timeframe='2020-07-01 2020-08-09',
     geo='AR')
     data = pytrend.interest_over_time()
     if not data.empty:
          data = data.drop(labels=['isPartial'],axis='columns')
          dataset.append(data)
result = pd.concat(dataset, axis=1)
result.to_csv('search_trends.csv')
executionTime = (time.time() - startTime)
print('Execution time in sec.: ' + str(executionTime))

KeyboardInterrupt: 